In [ ]:
with open('README.md', 'r') as f:
    count=0
    for line in f:
        if 'spark' in line.lower():
            count +=1

        
print(f"Masz {count} lini zawierających spark")

In [ ]:
import findspark
findspark.init('/Users/air/Desktop/spark/')

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("spark1")\
        .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

lines = sc.textFile('README.md')
count_s = lines.filter(lambda x: 'spark' in x.lower())
print(f"Masz {count_s.count()} lini zawierających spark")

In [ ]:
%%file spark1.py

from pyspark.sql import SparkSession

if __name__ == "__main__":
    
    spark = SparkSession.builder\
        .appName("spark1")\
        .getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("ERROR")
    lines = sc.textFile('README.md')
    spark_count = lines.filter(lambda x: 'spark' in x.lower())
    print(f"Masz {spark_count.count()} wierszy z wyrazem spark")

In [ ]:
! spark-submit spark1.py

In [ ]:
from pyspark.sql import Row
from pyspark.sql import functions as f

In [ ]:
person1 = Row(age=34, name='Adam')

In [ ]:
person1

In [ ]:
person1['name'], person1.age

In [ ]:
'name' in person1

In [ ]:
newPerson = Row("age","name")

In [ ]:
person2 = newPerson(24,"Alicja")

In [ ]:
person3 = newPerson(None, None)
person4 = newPerson(33, None)
person5 = newPerson(None, 'Peter')
person6 = newPerson(32, 'Peter')
person7 = newPerson(40, 'Greg')

In [ ]:
lista = [person1, person2, person3, person4, 
                                  person5, person6, person7]

In [ ]:
peopleDF = spark.createDataFrame(lista)

In [ ]:
peopleDF 

In [ ]:
peopleDF.show() 

In [ ]:
peopleDF.schema

In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

In [ ]:
schema = StructType([StructField("V1", IntegerType()), StructField("V2", StringType())])

In [ ]:
df = spark.createDataFrame([(1,2),(3,4),(5,6)], schema)

In [ ]:
df.show()

In [ ]:
df.rdd

In [ ]:
df.rdd.collect()

In [ ]:
df.rdd.toDF().show()

In [ ]:
%%file example.json

'''
{"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}}
{"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}}
{"string":"string3","int":3,"array":[3,6,9],"dict": {"key": "value3", "extra_key": "extra_value3"}}

In [ ]:
d = spark.read.json('example.json')

In [ ]:
d.printSchema()

In [ ]:
d.show()

In [ ]:
df = spark.read.csv("RDD_input")
df.show(4)

In [ ]:
col_names = ['line']
df = df.toDF(*col_names)
df.show(2)

In [ ]:
words = df.select(f.explode(f.split(df.line, " ")).alias("word"))

In [ ]:
words.show()

In [ ]:
import findspark
findspark.init('/Users/air/Desktop/spark/')

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession\
        .builder\
        .master("local[2]") \
        .appName("Stream_DF")\
        .getOrCreate()

In [ ]:
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
lines = spark.readStream\
        .format("socket")\
        .option("host", "localhost")\
        .option("port", 9999)\
        .load()

In [ ]:
words = lines.select(f.explode(f.split(lines.value, " ")).alias("word"))
wordCounts = words.groupBy("word").count()
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()
query.stop()


lines = DataFrame reprezentujący nieograniczoną tabelę zawierającą dane strumieniowe. 
Zawiera ona jedną kolumnę o nazwie `value`. Każda nowa linia to wiersz w tabeli. 

<img src="https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png"/>

In [ ]:
! spark-submit spark2.py